In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [1]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-kr2ej57t
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-kr2ej57t
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
     |████████████████████████████████| 129 kB 5.9 MB/s eta 0:00:01
     |████████████████████████████████| 54.7 MB 25.5 MB/s eta 0:00:01     |█████████████                   | 22.0 MB 38.4 MB/s eta 0:00:01     |█████████████████████████████   | 49.7 MB 28.7 MB/s eta 0:00:01
  Using cached onnxruntime-1.8.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.5 MB)
  Using cached sentencepiece-0.1.96-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
     |████████████████████████████████| 2.5 MB 32.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.7 MB 29.1 MB/s eta 0:00:01
     |████████████████████████████████| 73 kB 3.3 MB/s  eta 0:00:01


In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [2]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

2022-10-18 13:10:29.445731: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [4]:
##GPU 사용 시
device = torch.device("cuda:0")

In [5]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/mglee/VSCODE/git_folder/naver_comments/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/naver_comments/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [11]:
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

--2022-10-17 23:46:00--  https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.84.18, 2620:100:6034:18::a27d:5412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.84.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/374ftkec978br3d/ratings_train.txt [following]
--2022-10-17 23:46:01--  https://www.dropbox.com/s/dl/374ftkec978br3d/ratings_train.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucba262d33702ff268dd41ab3c9c.dl.dropboxusercontent.com/cd/0/get/Bu8Nt-S95m4yC1Y_TDEzloyjhE8G1TlH8kYNI2YHQBGJ3PXDz0L8Vj9M1EVyKu3zX2wGyMp3lXrKVNnFNYrju7DN4LHgv4o-6P8pBAFZxdoThYnL4--GbtRFQiPiZv33KIadbgmHDdobD-_1lB1k-5LoVbszgX_UdHE1Jg_H8-_PuA/file?dl=1# [following]
--2022-10-17 23:46:02--  https://ucba262d33702ff268dd41ab3c9c.dl.dropboxusercontent.com/cd/0/get/Bu8Nt-S95m4yC1Y_TDEzloyjhE8G1TlH8kYNI2YHQBGJ3PXDz0L8Vj9M1EVy

In [6]:
dataset_train = nlp.data.TSVDataset("ratings_train.txt?dl=1", field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("ratings_test.txt?dl=1", field_indices=[1,2], num_discard_samples=1)

In [7]:
dataset_train

In [8]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/mglee/VSCODE/git_folder/naver_comments/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [9]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [10]:
## Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 500
learning_rate =  5e-5

In [12]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [13]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [14]:
for data in train_dataloader:
  print("Data: ", data)
  print("Waveform: {}\nSample rate: {}\nLabels: {}".format(data[0], data[1], data[2]))
  break

Data:  [tensor([[   2, 3093, 1698,  ...,    1,    1,    1],
        [   2,  517, 7989,  ...,    1,    1,    1],
        [   2, 1458, 7191,  ...,    1,    1,    1],
        ...,
        [   2,  517, 6592,  ...,    1,    1,    1],
        [   2, 3255, 7728,  ...,    1,    1,    1],
        [   2,  517, 7265,  ...,    1,    1,    1]], dtype=torch.int32), tensor([15, 36, 14, 26, 47, 41, 16, 64, 13, 38, 17, 35, 29, 34, 34, 13, 45, 23,
        35, 27, 27, 10, 64, 17, 11, 42, 19,  9,  6, 36, 33, 25],
       dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
        0, 0, 1, 0, 1, 1, 1, 0], dtype=torch.int32)]
Waveform: tensor([[   2, 3093, 1698,  ...,    1,    1,    1],
        [   2,  517, 7989,  ...,    1,   

In [15]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [16]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [17]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [18]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [19]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [20]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [21]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [22]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_22753/1280546429.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/4688 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7215656638145447 train acc 0.4375
epoch 1 batch id 201 loss 0.7043008208274841 train acc 0.5203669154228856
epoch 1 batch id 401 loss 0.45091691613197327 train acc 0.6092581047381546
epoch 1 batch id 601 loss 0.4005853533744812 train acc 0.6649334442595674
epoch 1 batch id 801 loss 0.5966969728469849 train acc 0.701310861423221
epoch 1 batch id 1001 loss 0.401917964220047 train acc 0.7279595404595405
epoch 1 batch id 1201 loss 0.5256664752960205 train acc 0.7467995420482931
epoch 1 batch id 1401 loss 0.3270328640937805 train acc 0.7610635260528195
epoch 1 batch id 1601 loss 0.48829758167266846 train acc 0.7725054653341662
epoch 1 batch id 1801 loss 0.4375264048576355 train acc 0.7809203220433093
epoch 1 batch id 2001 loss 0.3588522672653198 train acc 0.787699900049975
epoch 1 batch id 2201 loss 0.6086718440055847 train acc 0.7943974329850069
epoch 1 batch id 2401 loss 0.4533323645591736 train acc 0.7992242815493544
epoch 1 batch id 2601 loss 0.1753716915845871

/tmp/ipykernel_22753/1280546429.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1563 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8956733845169546


  0%|          | 0/4688 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.4938872754573822 train acc 0.84375
epoch 5 batch id 201 loss 0.03021354414522648 train acc 0.972636815920398
epoch 5 batch id 401 loss 0.11897013336420059 train acc 0.9735037406483791
epoch 5 batch id 601 loss 0.011960766278207302 train acc 0.9744696339434277
epoch 5 batch id 801 loss 0.08929017931222916 train acc 0.9748361423220974
epoch 5 batch id 1001 loss 0.013419167138636112 train acc 0.9743693806193806
epoch 5 batch id 1201 loss 0.4733012914657593 train acc 0.9743442964196503
epoch 5 batch id 1401 loss 0.11880923807621002 train acc 0.9749286224125625
epoch 5 batch id 1601 loss 0.10578501224517822 train acc 0.9751717676452217
epoch 5 batch id 1801 loss 0.20203867554664612 train acc 0.9752568017767906
epoch 5 batch id 2001 loss 0.09572886675596237 train acc 0.9757777361319341
epoch 5 batch id 2201 loss 0.20463095605373383 train acc 0.9760478191731031
epoch 5 batch id 2401 loss 0.0053940205834805965 train acc 0.9763509995835069
epoch 5 batch id 2601 loss 0.

  0%|          | 0/1563 [00:00<?, ?it/s]

epoch 5 test acc 0.8967530390275112


In [23]:
torch.save(model, '/home/mglee/VSCODE/git_folder/BERT/Sentiment/KoBERT_Sentiment_e5_221018.pt')  # 전체 모델 저장
torch.save(model.state_dict(), '/home/mglee/VSCODE/git_folder/BERT/Sentiment/Kobert_Sentiment_e5_state_dict_221018.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, '/home/mglee/VSCODE/git_folder/BERT/Sentiment/Kobert_Sentiment_all.tar') 

In [ ]:
PATH = '../result/kobert/'

pred_model = torch.load(PATH + 'KoBERT_0705_e15_py.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
# pred_model.load_state_dict(torch.load(PATH + 'Kobert_0627_py_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

# checkpoint = torch.load(PATH + 'all.tar')   # dict 불러오기
# pred_model.load_state_dict(checkpoint['model'])
#optimizer.load_state_dict(checkpoint['optimizer'])

### TEST DATA

In [79]:
import pandas as pd

posco = pd.read_csv("../../naver_comments/result/posco_token.csv")

In [104]:
test = posco.loc[:,['comment']]

In [105]:
test['label'] = 0

In [106]:
test

,comment,label
0,"""즈그말은 그렇지만 현장분위기는 완전 틀리던디""",0
1,"""포스코 화이팅 !! 가즈아""",0
2,"""또 거짓말 . 언플할 시간에 복구에 집중해라""",0
3,"""이 기회에 중간유통 마진이랑 업체들 정리 하시고\n스틸트레이드 같은 형식의 플랫폼...",0
4,"""역대 최악 최정우 되고 나서 재해사고\n엄청 났지..그냥 너는 포스코랑 안 맞다\...",0
...,...,...
13506,"""아~~자정까지 기다렸네...태풍구경할라구~~~""",0
13507,"""방송만 난리부르스고 수도권은 느낌도 안나네""",0
13508,"""삼성은 전자만""",0
13509,"""그냥 일해라 태풍이 뭐시라고 겁먹노 귀가찬다.""",0


In [107]:
class BERTDataset_test(Dataset):
    def __init__(self, dataset,bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([" ".join(str(dataset.iloc[i]['comment']))]) for i in range(len(dataset))]
        self.labels = [np.int32(dataset.iloc[i]['label']) for i in range(len(dataset))]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))



In [108]:
comment_test = BERTDataset_test(test, tok, max_len, True, False)

In [109]:
comment_test

In [110]:
pred_dataloader = torch.utils.data.DataLoader(comment_test, batch_size = batch_size, num_workers = 8)

In [114]:
for data in pred_dataloader:
  print("Data: ", data)
  print("Waveform: {}\nSample rate: {}\nLabels: {}".format(data[0], data[1], data[2]))
  break

Data:  [tensor([[   2,  518,  517,  ...,    1,    1,    1],
        [   2,  518, 4856,  ...,    1,    1,    1],
        [   2,  518, 1861,  ...,    1,    1,    1],
        ...,
        [   2,  518, 3552,  ...,    1,    1,    1],
        [   2,  518, 4457,  ...,    0,  517,    3],
        [   2,  518, 2718,  ..., 3758, 1562,    3]], dtype=torch.int32), tensor([29, 20, 26, 64, 60, 18, 17, 64, 23, 45, 64, 35, 64, 28, 35, 40, 17, 64,
        64, 35, 42, 28, 64, 59, 64, 35, 24, 20, 64, 17, 17, 34, 64, 64, 64, 29,
        64, 64, 61, 64, 52, 29, 64, 64, 13, 64, 64, 21, 30, 23, 64, 34,  7,  8,
        17, 47, 64, 64, 61, 64, 52, 29, 64, 64], dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0,

In [115]:
out_lst = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(pred_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    out_lst.append(out.data.cpu())
    max_vals, max_indices = torch.max(out, 1)

100%|██████████| 212/212 [00:08<00:00, 24.51it/s]


In [132]:
pred = []
for batch in out_lst:
    for item in batch:
        pred.append(np.argmax(item.numpy()))

In [135]:
test['label'] = pred

In [145]:
posco['sentiment'] = pred

In [149]:
posco.groupby('sentiment').size()

sentiment
0    11930
1     1581
dtype: int64

In [148]:
posco.to_csv("../../naver_comments/result/naver_POSCO_sentiment.csv")